In [1]:
import math
import numpy as np
import sympy as sm
from sympy import Derivative, diff
import itertools
from itertools import groupby
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from tabulate import tabulate
from pprint import pprint
import TDMA

from IPython.display import display, Latex, clear_output

In [2]:
#sm.init_printing(use_latex='mathjax')
#x, y  = sm.symbols('x, y', real=True)
#c = sm.symbols("c")
#u = sm.Function('u')
#ux = u(x)
#ux1 = Derivative(u(x), x)
#ux2 = Derivative(u(x), (x, 2))

#h, ww_current, ww_left, ww_right, ww_up, ww_down = sm.symbols('h, u_ij, u_i_j+1, u_i_j-1, u_i-1_j, u_i+1_j')
#ww_current, ww_left, ww_right, ww_up, ww_down

In [3]:
zad1 = {
    'A': 200, 'B': 140,
    'R': 40, 'h': 4,
    'P': 65 * 10**9,
    'E': 140,
    'v': 0.28
}
zad1['D'] = zad1['E'] * zad1['h']**3 / (12 * (1 - zad1['v']**2))
zad1['R_left']  = zad1['A']/2 - zad1['R']
zad1['R_right'] = zad1['A']/2 + zad1['R']

In [4]:
n, m = 95, 95
hh = (zad1['A']) / (n-1)
ll = (zad1['B']) / (m-1)
list_x = [hh*i for i in range(n)]
list_y = [ll*i for i in range(m)]

hh, ll

(2.127659574468085, 1.4893617021276595)

In [5]:
#func_w0 = (ww_left + ww_right + ww_up + ww_down)/4 - zad1['P']*hh**2 / (4*zad1['D'])
#func_w = sm.Eq((-4*ww_current + ww_left + ww_right + ww_up + ww_down), zad1['P']*hh**2 / (zad1['D']))
halfCircle = lambda x: (zad1['R']**2 - (x - zad1['A']/2)**2)**0.5
#func_w

In [6]:
A_type = np.full((m,n), 1)
A_index = np.full((m,n), 1)

In [7]:
indexer = 0
for i in range(1, m - 1):
    for j in range(1, n - 1):
        #if(A[i][j][1] == 1):
        #    continue
        if (list_x[j] < zad1['R_left'] or list_x[j] > zad1['R_right']):
            A_type[i][j] = 0
            A_index[i][j] = indexer
            indexer += 1
            continue
        y_circle = halfCircle(list_x[j])
        y_cur = list_y[i]
        if (y_cur > y_circle):
            A_type[i][j] = 0
            A_index[i][j] = indexer
            indexer += 1
        else:
            A_type[i][j] = 2
            #A_symbols[i][j] = 0
    #break
A_axis_size = indexer
A_axis_size

7879

In [8]:
#print(np.array(A_type))
#print(np.array(A_index))

### С помощью Sympy(Х**ня)

### С помощью numpy.linalg.solve

In [9]:
A = np.zeros((A_axis_size,A_axis_size), dtype=np.float64)
list_f = [zad1['P']*hh**2 * ll**2 / (zad1['D']) for i in range(indexer)]
indexer = 0
for i in range(1, m - 1):
    for j in range(1, n - 1):
        if(A_type[i][j] == 2):
            continue
        A[indexer,int(A_index[i][j])] = -2 * (hh**2 + ll**2)
        if (A_type[i][j-1] == 0) :
            A[indexer,int(A_index[i][j-1])] = ll**2
        if (A_type[i][j+1] == 0):
            A[indexer,int(A_index[i][j+1])] = ll**2
        if (A_type[i-1][j] == 0):
            A[indexer,int(A_index[i-1][j])] = hh**2
        if (A_type[i+1][j] == 0):
            A[indexer,int(A_index[i+1][j])] = hh**2
        indexer += 1

In [13]:
A_solve = np.linalg.solve(A, list_f)

In [53]:
A_values = np.full((m,n), 0, dtype=np.int64)
indexer = 0
for i in range(1, m - 1):
    for j in range(1, n - 1):
        if(A_type[i][j] == 2):
            continue
        A_values[i][j] = A_solve[indexer]
        indexer += 1

In [18]:
lighting_effects = dict(ambient=0.4, diffuse=0.5, roughness = 0.9, specular=2, fresnel=0.2)
fig = go.Figure(go.Surface(
    #contours = {
    #    "x": {"show": True, "start": 1.5, "end": 2, "size": 0.04, "color":"white"},
    #    "z": {"show": True, "start": 0.5, "end": 0.8, "size": 0.05}
    #},
    x = list_x,
    y = list_y,
    z = A_values,
    colorscale='Viridis', lighting=lighting_effects
))
fig.update_layout(
        scene = {
            #"xaxis": { "nticks": 10, 
            #            "gridcolor":'rgb(255, 255, 255)',
            #            "zerolinecolor":'rgb(255, 255, 255)',
            #            "showbackground":True,
            #            "backgroundcolor":'rgb(230, 230,230)'},
            #"yaxis": {"nticks": 10,
            #            "gridcolor":'rgb(255, 255, 255)',
            #            "zerolinecolor":'rgb(255, 255, 255)',
            #            "showbackground":True,
            #            "backgroundcolor":'rgb(230, 230,230)'},
            #"zaxis": {"nticks": 4,
            #            "gridcolor":'rgb(255, 255, 255)',
            #            "zerolinecolor":'rgb(255, 255, 255)',
            #            "showbackground":True,
            #            "backgroundcolor":'rgb(230, 230,230)'},
            'camera_eye': {"x": -1, "y": 0.5, "z": 1},
            "aspectratio": {"x": 1, "y": 1, "z": 1}
        })
fig.show()